In [1]:
# !pip install dlib
import cv2
import numpy as np
import pandas as pd
import os
from pathlib import Path
from math import radians, degrees
import mediapipe as mp

In [19]:
# --- CONFIG ---
VIDEO_DIR = "C:\\cv_project\\data\\Videos"
OUTPUT_DIR = "C:\\cv_project\\code\\output_csv"
FEATURE_COLS = [
    "Pitch", "Yaw", "Roll", "inBrL", "otBrL", "inBrR", "otBrR",
    "EyeOL", "EyeOR", "oLipH", "iLipH", "LipCDt"
]

In [20]:
from mediapipe.tasks.python import vision
from mediapipe.tasks.python import BaseOptions

base_options = BaseOptions(model_asset_path='face_landmarker.task', delegate="CPU")
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                        output_face_blendshapes=True,
                                        output_facial_transformation_matrixes=True,
                                        num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

In [27]:
cap = cv2.VideoCapture("C:\\cv_project\\data\\Videos\\P1.avi")
frames = []
ret = True
while ret:
   ret, frame = cap.read()
   if not ret:
       break
   frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
   frames.append(frame)

In [29]:
frame_idx = 1700
image = mp.Image(
   image_format=mp.ImageFormat.SRGB, data=frames[frame_idx]
)
detection_result = detector.detect(image)
frame_scores = np.array([blendshape.score for blendshape in detection_result.face_blendshapes[0]])
blendshape_names = [blendshape.category_name for blendshape in detection_result.face_blendshapes[0]]

In [30]:
blendshape_names

['_neutral',
 'browDownLeft',
 'browDownRight',
 'browInnerUp',
 'browOuterUpLeft',
 'browOuterUpRight',
 'cheekPuff',
 'cheekSquintLeft',
 'cheekSquintRight',
 'eyeBlinkLeft',
 'eyeBlinkRight',
 'eyeLookDownLeft',
 'eyeLookDownRight',
 'eyeLookInLeft',
 'eyeLookInRight',
 'eyeLookOutLeft',
 'eyeLookOutRight',
 'eyeLookUpLeft',
 'eyeLookUpRight',
 'eyeSquintLeft',
 'eyeSquintRight',
 'eyeWideLeft',
 'eyeWideRight',
 'jawForward',
 'jawLeft',
 'jawOpen',
 'jawRight',
 'mouthClose',
 'mouthDimpleLeft',
 'mouthDimpleRight',
 'mouthFrownLeft',
 'mouthFrownRight',
 'mouthFunnel',
 'mouthLeft',
 'mouthLowerDownLeft',
 'mouthLowerDownRight',
 'mouthPressLeft',
 'mouthPressRight',
 'mouthPucker',
 'mouthRight',
 'mouthRollLower',
 'mouthRollUpper',
 'mouthShrugLower',
 'mouthShrugUpper',
 'mouthSmileLeft',
 'mouthSmileRight',
 'mouthStretchLeft',
 'mouthStretchRight',
 'mouthUpperUpLeft',
 'mouthUpperUpRight',
 'noseSneerLeft',
 'noseSneerRight']

## Facial Feature Extraction

In [ ]:
def process_frames(frames, detector):
    """Process frames to calculate blendshape scores."""
    blendshape_sums = None
    frame_count = 0
    blendshape_names = []

    for frame in frames:
        image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        detection_result = detector.detect(image)

        # Skip frames without face_blendshapes
        if not detection_result.face_blendshapes:
            continue

        frame_scores = np.array([blendshape.score for blendshape in detection_result.face_blendshapes[0]])
        if blendshape_sums is None:
            blendshape_sums = np.zeros_like(frame_scores)
            blendshape_names = [blendshape.category_name for blendshape in detection_result.face_blendshapes[0]]
        blendshape_sums += frame_scores
        frame_count += 1

    return blendshape_sums, frame_count, blendshape_names


def calculate_average_blendshapes(blendshape_sums, frame_count, blendshape_names):
    """Calculate average blendshapes for a video."""
    if frame_count > 0:
        blendshape_averages = blendshape_sums / frame_count
        return dict(zip(blendshape_names, blendshape_averages))
    else:
        return None


In [34]:
##Labels

def process_video(video_file, detector, output_dir):
    """Process a single video file."""
    print(f"Processing video: {video_file.name}")

    # Open the video file and read frames
    cap = cv2.VideoCapture(str(video_file))
    frames = []
    ret = True
    while ret:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
    cap.release()

    # Process frames and calculate blendshape averages
    blendshape_sums, frame_count, blendshape_names = process_frames(frames, detector)
    video_data = calculate_average_blendshapes(blendshape_sums, frame_count, blendshape_names)

    if video_data:
        # Ensure video name is the first column and columns are set using blendshape_names
        video_data_row = {"Video": video_file.name}
        video_data_row.update({name: video_data.get(name, 0) for name in blendshape_names})
        return video_data_row
    else:
        print(f"No face blendshapes detected in video: {video_file.name}")
        return None


def save_results_to_csv(video_data, output_dir):
    """Save video data to a CSV file."""
    output_file = Path(output_dir) / "results.csv"
    df = pd.DataFrame([video_data])
    if not output_file.exists():
        df.to_csv(output_file, index=False)
    else:
        df.to_csv(output_file, mode='a', header=False, index=False)


def main(video_dir, output_dir, detector):
    """Main function to process all videos in the directory."""
    for video_file in sorted(Path(video_dir).glob("*.avi")):  # Sort files alphabetically
        video_data = process_video(video_file, detector, output_dir)
        if video_data:
            save_results_to_csv(video_data, output_dir)